# Tensorflow 로 LSTM 만들기

- tensorflow 를 사용하여 LSTM 을 만드는 방법 소개

* 사용되는 input(data) 와 output(label) 종류
    - input  : 길이 20의 2진수, ex) 00010011111111111111
    - output : 길이 20의 2진수, ex) 00010100000000000000
    - output 은 input 에 1을 더한 값
        - 00010011111111111111 + 1 = 00010100000000000000

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer

## 1. Data Generation & Preprocessing

### 1-1. 길이 20의 2진수 문자열 2^20 개를 생성

* data_size : data 의 크기, 2^20
* train_set_size : data_set 에서 train 으로 사용될 data 의 양, 2^18

In [3]:
data_size = 2**20
train_set_size = 2**18

data_set = ['{0:020b}'.format(i) for i in range(data_size)]

In [4]:
print('* 생성된 데이터 표본')
for data in data_set[:10]:
    print(data)

* 생성된 데이터 표본
00000000000000000000
00000000000000000001
00000000000000000010
00000000000000000011
00000000000000000100
00000000000000000101
00000000000000000110
00000000000000000111
00000000000000001000
00000000000000001001


### 1-2. 길이 20의 배열로 분할하기

2진수 문자열을 LSTM 에 적용하기 위하여 길이 20의 배열로 변환한다

* 문자열을 크기20의 배열 [20] 으로 분할한다.

In [5]:
ti  = []
for i in data_set:
    temp_list = []
    for j in i:
            temp_list.append(j)
    ti.append(temp_list)
temp_data_set = ti

In [6]:
print('* 분할된 데이터 표본')
for origin, data in zip(data_set[:3], temp_data_set[:3]):
    print(origin, '\n\t -> ', data)

* 분할된 데이터 표본
00000000000000000000 
	 ->  ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
00000000000000000001 
	 ->  ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1']
00000000000000000010 
	 ->  ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0']


### 1-3. data 를 input 과 output 으로 나누기

* output = input + 1 이므로, input 과 output 의 크기는 2^20 - 1 이다.
    * input 범위 : 0 ~ 2^20-1
    * output 범위 : 1 ~ 2^20

In [7]:
temp_data = []
temp_target = []
for idx in range(data_size - 1):
    data = [int(val) for val in temp_data_set[idx][::-1]]
    target = [int(val) for val in temp_data_set[idx+1][::-1]]
    
    temp_data.append(data)
    temp_target.append(target)

In [8]:
temp_data = np.array(temp_data)
temp_target = np.array(temp_target)

* 시계열 훈련을 위해 배열을 역순으로 반전 시킨후 저장
* 이 과정이 없다면, 미래를 보고 과거를 맞추는 셈

In [9]:
for data, label in zip(temp_data[:3], temp_target[:3]):
    print('전 : ', data,', 후 : ', label)

전 :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] , 후 :  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
전 :  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] , 후 :  [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
전 :  [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] , 후 :  [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### 1-4. shuffle 및 train, test 로 분리하기

* shuffle 이 더 효율적

In [10]:
shuffle = True

if shuffle == True:
    indices = np.random.permutation(range(data_size-1))

    temp_data = temp_data[indices]
    temp_target = temp_target[indices]

In [11]:
test_input = temp_data[train_set_size:]
test_output = temp_target[train_set_size:]
train_input = temp_data[:train_set_size]
train_output = temp_target[:train_set_size]

## 2. LSTM 만들기

#### 2-1. 변수 생성

* batch_size : input 으로 들어가는 배열의 크기, 2^9
* step_size : 시계열의 길이, 20
* class_size : output 의 label 종류, 0 or 1 - 2개
* num_hidden : 한 cell 의 state 의 크기, 32
* dropout_probs : dropout 확률, 1, 데이터의 특성상 한자리의 값도 빠지면 예측이 불가능하여 1로 설정

In [12]:
batch_size = 2**9
step_size = 20
class_size = 2
num_hidden = 32

dropout_probs = 1.0

* placeholder : memory <-> graphic card, 사이의 데이터를 전송해주는 bridge 개념
* x, y 를 통하여 그래픽카드 메모리에 input 과 output 을 전달할 수 있음

In [13]:
x = tf.placeholder(tf.int32, [None, step_size])
y = tf.placeholder(tf.int32, [None, step_size])

* embedding : word2vector 의 역할을 하는 함수
    * [512, 20] -> [512, 20, 10]

In [14]:
vector_size = 10

embeddings = tf.get_variable('embedding_matrix', [class_size, vector_size])
rnn_inputs = tf.nn.embedding_lookup(embeddings, x)

In [15]:
print('* embedding 처리 전의 input 모습')
print(x.shape)
print('* embedding 처리 후의 input 모습')
print(rnn_inputs.shape)

* embedding 처리 전의 input 모습
(?, 20)
* embedding 처리 후의 input 모습
(?, 20, 10)


#### 2-2. LSTM Cell 생성

* 32 크기의 state 를 가지는 LSTM Cell 생성

<img src = "single rnn cell.png">

In [16]:
cell = tf.contrib.rnn.LSTMCell(num_hidden, state_is_tuple=True)

* input 에 dropout 을 걸어준다

In [17]:
cell = tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=dropout_probs)

#### 2-3. MultiLayer RNN

* Cell 을 중첩하여 구성한다.

<img src = "multi rnn cell.png">

In [18]:
num_layers = 2

cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers, state_is_tuple=True)
cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=dropout_probs)

In [19]:
outputs, _ = tf.nn.dynamic_rnn(cell, rnn_inputs, dtype=tf.float32)

### 3. output 가공

* 현재까지 기본이 되는 LSTM 을 구성하였다.
* 앞으로는 Cell 을 구성하여 출력된 output 과 loss 를 어떻게 가공하느냐에 따라서, N to 1 혹은 N to N 으로 만들수 있다.

<img src = "rnn types.png">

                                                    ▲

* 앞으로의 코드는 "many to one" 의 RNN Cell 구성 방법이다.

#### 3-1. state 를 output 으로 만들기

* 32 개의 output state 를 "0 or 1" 의 20개 배열로 만들기 위하여, [32, 20 * 2] 의 weight 와 [20 * 2] 의 bias 를 생성

In [20]:
weight = tf.Variable(tf.truncated_normal([num_hidden, step_size * class_size]))
bias = tf.Variable(tf.constant(0.1, shape=[step_size * class_size]))

* 현재의 output 의 크기
* [512, 20, 32]

* outputs 에서 마지막 state 만 output 으로 사용하기 위해 크기를 변경한다
    * [512, 20, 32] -> [20, 512, 32]
* outputs 의 마지막 state 만 사용한다

In [21]:
outputs = tf.transpose(outputs, [1, 0, 2])

last_state = outputs[-1]

* [512, 32] 의 last_state 만 연산한다
* 연산된 logits 를 [10240, 2] 의 크기로 변경한다

In [22]:
logits = tf.matmul(last_state, weight) + bias
logits = tf.reshape(logits, [-1, class_size])

* 변경된 output 의 모습
* [512, 32] -> [512, 40] -> [10240, 2]

#### 3-2. loss 선언

* [512, 20] 의 output 을 logits 의 크기에 맞춰 변경 -> [10240, 1]
* logits 와 label 간의 cross_entropy 를 계산해주는 함수를 이용하여 total_loss 를 연산

In [23]:
y_as_list = tf.reshape(y, [-1])
total_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y_as_list))

#### 3-3. prediction & accuracy 선언

* logits 을 softmax 처리후 argmax 함수를 이용하여, [10240, 1] 로 만든다.
* [10240, 1] 크기의 prediction 과 output 을 [512, 20] 로 만든다.
* 비교함수를 이용하여 정확도를 계산

In [24]:
predictions = tf.argmax(tf.nn.softmax(logits), 1)
predictions = tf.reshape(predictions, [-1, step_size])

target = tf.reshape(y_as_list, [-1, step_size])

matching = tf.reduce_all(tf.equal(predictions, tf.cast(target, tf.int64)), axis=1)

accuracy = tf.reduce_mean(tf.cast(matching, dtype=tf.float32))

#### 3-4. training algorithm 선택

In [25]:
optimizer = tf.train.AdagradOptimizer(0.1)
train_step = optimizer.minimize(total_loss)

In [26]:
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

* 100 번의 epoch 를 사용하여 훈련, 정확도가 100% 가 되면 훈련 종료

In [27]:
no_of_batches = int(len(train_input)/batch_size)
epoch = 100
for i in range(epoch):
    ptr = 0
    avg_acc = 0
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        _, lss, acc = sess.run([train_step, total_loss, accuracy], feed_dict={x: inp, y: out})
        avg_acc += acc / no_of_batches
        
    print("Epoch - ",str(i), ', loss : %.3e' % lss, ', acc : %.3f' % (avg_acc * 100))
       
    if avg_acc == 1:
        break

Epoch -  0 , loss : 6.059e-01 , acc : 0.002
Epoch -  1 , loss : 5.442e-01 , acc : 0.007
Epoch -  2 , loss : 4.710e-01 , acc : 0.106
Epoch -  3 , loss : 4.300e-01 , acc : 0.428
Epoch -  4 , loss : 3.513e-01 , acc : 1.449
Epoch -  5 , loss : 3.049e-01 , acc : 3.798
Epoch -  6 , loss : 2.742e-01 , acc : 7.110
Epoch -  7 , loss : 2.297e-01 , acc : 12.664
Epoch -  8 , loss : 2.324e-01 , acc : 22.085
Epoch -  9 , loss : 1.609e-01 , acc : 36.193
Epoch -  10 , loss : 1.165e-01 , acc : 50.917
Epoch -  11 , loss : 8.835e-02 , acc : 62.770
Epoch -  12 , loss : 7.257e-02 , acc : 71.006
Epoch -  13 , loss : 1.663e-01 , acc : 79.392
Epoch -  14 , loss : 4.213e-02 , acc : 87.564
Epoch -  15 , loss : 3.057e-02 , acc : 94.948
Epoch -  16 , loss : 2.582e-02 , acc : 94.822
Epoch -  17 , loss : 2.207e-02 , acc : 97.892
Epoch -  18 , loss : 1.966e-02 , acc : 97.478
Epoch -  19 , loss : 1.774e-02 , acc : 97.663
Epoch -  20 , loss : 1.608e-02 , acc : 98.257
Epoch -  21 , loss : 1.487e-02 , acc : 97.919
Epoch

#### 3-5. Validation

In [28]:
count = 0
incorrect_idx_array = []
for idx in range(int(np.ceil(len(test_input) / batch_size))):
    s = idx * batch_size
    e = s + batch_size
    
    match = sess.run(matching, feed_dict={x: test_input[s:e], y: test_output[s:e]})
    
    count += np.sum(match)
    incorrect_idx_array = np.concatenate([incorrect_idx_array, np.where(match == False)[0] + s]).astype(int)
    
print('test set size: %d' % (len(test_input)), ', correct count: %d' % count)

test set size: 786431 , correct count: 786372


* 실패한 data set 확인

In [29]:
if len(incorrect_idx_array) != 0:
    inp, out = test_input[incorrect_idx_array], test_output[incorrect_idx_array]
else:
    inp, out = [[1] + [0] * 19, [0] + [1] * 19], [[0, 1] + [0] * 18, [1] * 20]
    
    
pred, match = sess.run([predictions, matching], feed_dict={x: inp, y: out})

print('='*30)
for a, b, c, d in zip(inp, out, pred, match):
    print('input     : ', end='')
    for o in a[::-1]:
        print(o, end='')
    print()
    print('output    : ', end='')
    for o in b[::-1]:
        print(o, end='')
    print()
    print('prediction: ', end='')
    for o in c[::-1]:
        print(o, end='')
    print()
    print('correct: %r' % d)
    print()
print('='*30)

input     : 01010010111111111111
output    : 01010011000000000000
prediction: 11010011000000000000
correct: False

input     : 00011101001111111111
output    : 00011101010000000000
prediction: 00011101011000000000
correct: False

input     : 01110011111111111111
output    : 01110100000000000000
prediction: 01110110000000000000
correct: False

input     : 01111111111111111111
output    : 10000000000000000000
prediction: 11111000000001010000
correct: False

input     : 11111000000111111111
output    : 11111000001000000000
prediction: 11111000001000010000
correct: False

input     : 01110111111111111111
output    : 01111000000000000000
prediction: 00111000000000000000
correct: False

input     : 11110111111111111111
output    : 11111000000000000000
prediction: 10111000000000000000
correct: False

input     : 10000011111111111111
output    : 10000100000000000000
prediction: 10000110000000000000
correct: False

input     : 10101111111111111111
output    : 10110000000000000000
prediction: 10